In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "metadata": {
    "ExecuteTime": {
     "end_time": "2024-11-30T20:52:42.824895Z",
     "start_time": "2024-11-30T20:52:42.821448Z"
    }
   },
   "source": [
    "from google.cloud import bigquery\n",
    "import warnings\n",
    "warnings.filterwarnings('ignore')\n",
    "\n",
    "# Initialize BigQuery Client\n",
    "client = bigquery.Client.from_service_account_json(\"secrets/serviceKey.json\")\n",
    "\n",
    "project_id = \"idmpproject-441123\"\n",
    "dataset_id = \"uberFareEstimation\"\n",
    "uber_table_id = 'uber_data'\n",
    "weather_table_id = 'weather_data'"
   ],
   "outputs": [],
   "execution_count": 5
  },
  {
   "cell_type": "code",
   "metadata": {
    "ExecuteTime": {
     "end_time": "2024-11-30T20:53:12.802877Z",
     "start_time": "2024-11-30T20:52:44.432327Z"
    }
   },
   "source": [
    "table_path = f\"{project_id}.{dataset_id}.{uber_table_id}\"\n",
    "\n",
    "# Define the query\n",
    "query = f\"\"\"\n",
    "    SELECT *\n",
    "    FROM `{table_path}`\n",
    "\"\"\"\n",
    "\n",
    "# Run the query\n",
    "query_job = client.query(query)\n",
    "\n",
    "# Wait for the query to complete and fetch results\n",
    "results = query_job.result()\n",
    "\n",
    "uber_df = results.to_dataframe()\n",
    "\n",
    "\n",
    "\n",
    "table_path = f\"{project_id}.{dataset_id}.{weather_table_id}\"\n",
    "\n",
    "# Define the query\n",
    "query = f\"\"\"\n",
    "    SELECT *\n",
    "    FROM `{table_path}`\n",
    "\"\"\"\n",
    "\n",
    "# Run the query\n",
    "query_job = client.query(query)\n",
    "\n",
    "# Wait for the query to complete and fetch results\n",
    "results = query_job.result()\n",
    "\n",
    "weather_df = results.to_dataframe()"
   ],
   "outputs": [],
   "execution_count": 6
  },
  {
   "cell_type": "code",
   "metadata": {
    "ExecuteTime": {
     "end_time": "2024-11-30T20:53:18.342994Z",
     "start_time": "2024-11-30T20:53:18.340697Z"
    }
   },
   "source": [
    "print(uber_df.shape)\n",
    "print(weather_df.shape)"
   ],
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "(330568, 9)\n",
      "(4147, 8)\n"
     ]
    }
   ],
   "execution_count": 8
  },
  {
   "cell_type": "code",
   "metadata": {
    "ExecuteTime": {
     "end_time": "2024-11-30T20:53:19.690477Z",
     "start_time": "2024-11-30T20:53:19.679671Z"
    }
   },
   "source": [
    "uber_df"
   ],
   "outputs": [
    {
     "data": {
      "text/plain": [
       "        distance cab_type          time_stamp              destination  \\\n",
       "0           0.94     Uber 2018-11-28 23:30:00                North End   \n",
       "1           0.94     Uber 2018-12-14 19:30:00                North End   \n",
       "2           0.63     Uber 2018-11-27 21:30:00       Financial District   \n",
       "3           0.63     Uber 2018-12-15 15:00:00       Financial District   \n",
       "4           0.63     Uber 2018-12-15 13:30:00       Financial District   \n",
       "...          ...      ...                 ...                      ...   \n",
       "330563      7.46     Uber 2018-12-02 12:30:00  Northeastern University   \n",
       "330564      3.45     Uber 2018-11-28 14:00:00            South Station   \n",
       "330565      0.62     Uber 2018-12-17 17:00:00                 West End   \n",
       "330566      3.61     Uber 2018-11-29 20:00:00                 Back Bay   \n",
       "330567      5.56     Uber 2018-12-14 17:30:00        Boston University   \n",
       "\n",
       "                    source  price  surge_multiplier      id       name  \n",
       "0            North Station    4.5               1.0   39765   UberPool  \n",
       "1            North Station    4.5               1.0  437984   UberPool  \n",
       "2            South Station    4.5               1.0    1644   UberPool  \n",
       "3            South Station    4.5               1.0   10780   UberPool  \n",
       "4            South Station    4.5               1.0   21598   UberPool  \n",
       "...                    ...    ...               ...     ...        ...  \n",
       "330563  Financial District   59.5               1.0  584592  Black SUV  \n",
       "330564            Back Bay   61.5               1.0  372302  Black SUV  \n",
       "330565    Haymarket Square   61.5               1.0  260720  Black SUV  \n",
       "330566           North End   67.0               1.0  520519  Black SUV  \n",
       "330567  Financial District   87.0               1.0   34184  Black SUV  \n",
       "\n",
       "[330568 rows x 9 columns]"
      ],
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>distance</th>\n",
       "      <th>cab_type</th>\n",
       "      <th>time_stamp</th>\n",
       "      <th>destination</th>\n",
       "      <th>source</th>\n",
       "      <th>price</th>\n",
       "      <th>surge_multiplier</th>\n",
       "      <th>id</th>\n",
       "      <th>name</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>0.94</td>\n",
       "      <td>Uber</td>\n",
       "      <td>2018-11-28 23:30:00</td>\n",
       "      <td>North End</td>\n",
       "      <td>North Station</td>\n",
       "      <td>4.5</td>\n",
       "      <td>1.0</td>\n",
       "      <td>39765</td>\n",
       "      <td>UberPool</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>0.94</td>\n",
       "      <td>Uber</td>\n",
       "      <td>2018-12-14 19:30:00</td>\n",
       "      <td>North End</td>\n",
       "      <td>North Station</td>\n",
       "      <td>4.5</td>\n",
       "      <td>1.0</td>\n",
       "      <td>437984</td>\n",
       "      <td>UberPool</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>0.63</td>\n",
       "      <td>Uber</td>\n",
       "      <td>2018-11-27 21:30:00</td>\n",
       "      <td>Financial District</td>\n",
       "      <td>South Station</td>\n",
       "      <td>4.5</td>\n",
       "      <td>1.0</td>\n",
       "      <td>1644</td>\n",
       "      <td>UberPool</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>0.63</td>\n",
       "      <td>Uber</td>\n",
       "      <td>2018-12-15 15:00:00</td>\n",
       "      <td>Financial District</td>\n",
       "      <td>South Station</td>\n",
       "      <td>4.5</td>\n",
       "      <td>1.0</td>\n",
       "      <td>10780</td>\n",
       "      <td>UberPool</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>0.63</td>\n",
       "      <td>Uber</td>\n",
       "      <td>2018-12-15 13:30:00</td>\n",
       "      <td>Financial District</td>\n",
       "      <td>South Station</td>\n",
       "      <td>4.5</td>\n",
       "      <td>1.0</td>\n",
       "      <td>21598</td>\n",
       "      <td>UberPool</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>...</th>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>330563</th>\n",
       "      <td>7.46</td>\n",
       "      <td>Uber</td>\n",
       "      <td>2018-12-02 12:30:00</td>\n",
       "      <td>Northeastern University</td>\n",
       "      <td>Financial District</td>\n",
       "      <td>59.5</td>\n",
       "      <td>1.0</td>\n",
       "      <td>584592</td>\n",
       "      <td>Black SUV</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>330564</th>\n",
       "      <td>3.45</td>\n",
       "      <td>Uber</td>\n",
       "      <td>2018-11-28 14:00:00</td>\n",
       "      <td>South Station</td>\n",
       "      <td>Back Bay</td>\n",
       "      <td>61.5</td>\n",
       "      <td>1.0</td>\n",
       "      <td>372302</td>\n",
       "      <td>Black SUV</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>330565</th>\n",
       "      <td>0.62</td>\n",
       "      <td>Uber</td>\n",
       "      <td>2018-12-17 17:00:00</td>\n",
       "      <td>West End</td>\n",
       "      <td>Haymarket Square</td>\n",
       "      <td>61.5</td>\n",
       "      <td>1.0</td>\n",
       "      <td>260720</td>\n",
       "      <td>Black SUV</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>330566</th>\n",
       "      <td>3.61</td>\n",
       "      <td>Uber</td>\n",
       "      <td>2018-11-29 20:00:00</td>\n",
       "      <td>Back Bay</td>\n",
       "      <td>North End</td>\n",
       "      <td>67.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>520519</td>\n",
       "      <td>Black SUV</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>330567</th>\n",
       "      <td>5.56</td>\n",
       "      <td>Uber</td>\n",
       "      <td>2018-12-14 17:30:00</td>\n",
       "      <td>Boston University</td>\n",
       "      <td>Financial District</td>\n",
       "      <td>87.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>34184</td>\n",
       "      <td>Black SUV</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "<p>330568 rows × 9 columns</p>\n",
       "</div>"
      ]
     },
     "execution_count": 9,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "execution_count": 9
  },
  {
   "cell_type": "code",
   "execution_count": 8,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>temp</th>\n",
       "      <th>location</th>\n",
       "      <th>clouds</th>\n",
       "      <th>pressure</th>\n",
       "      <th>rain</th>\n",
       "      <th>time_stamp</th>\n",
       "      <th>humidity</th>\n",
       "      <th>wind</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>36.26</td>\n",
       "      <td>West End</td>\n",
       "      <td>0.00</td>\n",
       "      <td>1012.37</td>\n",
       "      <td>0.0</td>\n",
       "      <td>2018-11-29 18:30:00</td>\n",
       "      <td>0.69</td>\n",
       "      <td>6.84</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>29.95</td>\n",
       "      <td>Beacon Hill</td>\n",
       "      <td>0.00</td>\n",
       "      <td>1015.19</td>\n",
       "      <td>0.0</td>\n",
       "      <td>2018-12-14 18:30:00</td>\n",
       "      <td>0.85</td>\n",
       "      <td>2.43</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>29.85</td>\n",
       "      <td>Boston University</td>\n",
       "      <td>0.00</td>\n",
       "      <td>1015.20</td>\n",
       "      <td>0.0</td>\n",
       "      <td>2018-11-27 11:30:00</td>\n",
       "      <td>0.85</td>\n",
       "      <td>2.33</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>29.83</td>\n",
       "      <td>Fenway</td>\n",
       "      <td>0.00</td>\n",
       "      <td>1015.20</td>\n",
       "      <td>0.0</td>\n",
       "      <td>2018-12-16 04:30:00</td>\n",
       "      <td>0.85</td>\n",
       "      <td>2.35</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>29.99</td>\n",
       "      <td>Haymarket Square</td>\n",
       "      <td>0.00</td>\n",
       "      <td>1015.19</td>\n",
       "      <td>0.0</td>\n",
       "      <td>2018-12-17 06:30:00</td>\n",
       "      <td>0.85</td>\n",
       "      <td>2.46</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>...</th>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4142</th>\n",
       "      <td>44.50</td>\n",
       "      <td>Financial District</td>\n",
       "      <td>0.78</td>\n",
       "      <td>1000.87</td>\n",
       "      <td>0.0</td>\n",
       "      <td>2018-11-26 07:00:00</td>\n",
       "      <td>0.96</td>\n",
       "      <td>3.58</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4143</th>\n",
       "      <td>44.48</td>\n",
       "      <td>North End</td>\n",
       "      <td>0.78</td>\n",
       "      <td>1000.87</td>\n",
       "      <td>0.0</td>\n",
       "      <td>2018-11-27 22:00:00</td>\n",
       "      <td>0.96</td>\n",
       "      <td>3.55</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4144</th>\n",
       "      <td>44.38</td>\n",
       "      <td>North Station</td>\n",
       "      <td>0.78</td>\n",
       "      <td>1000.87</td>\n",
       "      <td>0.0</td>\n",
       "      <td>2018-11-30 12:30:00</td>\n",
       "      <td>0.96</td>\n",
       "      <td>3.51</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4145</th>\n",
       "      <td>30.25</td>\n",
       "      <td>Beacon Hill</td>\n",
       "      <td>0.78</td>\n",
       "      <td>1035.03</td>\n",
       "      <td>0.0</td>\n",
       "      <td>2018-12-01 07:30:00</td>\n",
       "      <td>0.68</td>\n",
       "      <td>2.32</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4146</th>\n",
       "      <td>36.27</td>\n",
       "      <td>Financial District</td>\n",
       "      <td>0.78</td>\n",
       "      <td>1001.88</td>\n",
       "      <td>0.0</td>\n",
       "      <td>2018-12-14 02:30:00</td>\n",
       "      <td>0.65</td>\n",
       "      <td>10.93</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "<p>4147 rows × 8 columns</p>\n",
       "</div>"
      ],
      "text/plain": [
       "       temp            location  clouds  pressure  rain          time_stamp  \\\n",
       "0     36.26            West End    0.00   1012.37   0.0 2018-11-29 18:30:00   \n",
       "1     29.95         Beacon Hill    0.00   1015.19   0.0 2018-12-14 18:30:00   \n",
       "2     29.85   Boston University    0.00   1015.20   0.0 2018-11-27 11:30:00   \n",
       "3     29.83              Fenway    0.00   1015.20   0.0 2018-12-16 04:30:00   \n",
       "4     29.99    Haymarket Square    0.00   1015.19   0.0 2018-12-17 06:30:00   \n",
       "...     ...                 ...     ...       ...   ...                 ...   \n",
       "4142  44.50  Financial District    0.78   1000.87   0.0 2018-11-26 07:00:00   \n",
       "4143  44.48           North End    0.78   1000.87   0.0 2018-11-27 22:00:00   \n",
       "4144  44.38       North Station    0.78   1000.87   0.0 2018-11-30 12:30:00   \n",
       "4145  30.25         Beacon Hill    0.78   1035.03   0.0 2018-12-01 07:30:00   \n",
       "4146  36.27  Financial District    0.78   1001.88   0.0 2018-12-14 02:30:00   \n",
       "\n",
       "      humidity   wind  \n",
       "0         0.69   6.84  \n",
       "1         0.85   2.43  \n",
       "2         0.85   2.33  \n",
       "3         0.85   2.35  \n",
       "4         0.85   2.46  \n",
       "...        ...    ...  \n",
       "4142      0.96   3.58  \n",
       "4143      0.96   3.55  \n",
       "4144      0.96   3.51  \n",
       "4145      0.68   2.32  \n",
       "4146      0.65  10.93  \n",
       "\n",
       "[4147 rows x 8 columns]"
      ]
     },
     "execution_count": 8,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "weather_df"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {
    "ExecuteTime": {
     "end_time": "2024-11-30T20:53:24.963654Z",
     "start_time": "2024-11-30T20:53:24.961115Z"
    }
   },
   "source": [
    "uber_df.rename(columns={\"time_stamp\": \"uber_time_stamp\"}, inplace=True)\n",
    "weather_df.rename(columns={\"time_stamp\": \"weather_time_stamp\"}, inplace=True)"
   ],
   "outputs": [],
   "execution_count": 11
  },
  {
   "cell_type": "code",
   "metadata": {
    "ExecuteTime": {
     "end_time": "2024-11-30T20:53:25.920188Z",
     "start_time": "2024-11-30T20:53:25.806119Z"
    }
   },
   "source": [
    "import pandas as pd\n",
    "\n",
    "# Perform the join\n",
    "joined_df = pd.merge(\n",
    "    uber_df,\n",
    "    weather_df,\n",
    "    left_on=[\"uber_time_stamp\", \"source\"],\n",
    "    right_on=[\"weather_time_stamp\", \"location\"],\n",
    "    how=\"left\"  \n",
    ")\n",
    "\n",
    "# Replace missing values in 'location' with the values from 'source'\n",
    "joined_df['location'] = joined_df['location'].fillna(joined_df['source'])\n",
    "\n",
    "# Ensure the DataFrame is sorted by location and timestamp\n",
    "joined_df = joined_df.sort_values(by=['location', 'uber_time_stamp'])\n",
    "\n",
    "# Display the result\n",
    "joined_df.head(5)\n"
   ],
   "outputs": [
    {
     "data": {
      "text/plain": [
       "        distance cab_type     uber_time_stamp              destination  \\\n",
       "254787      6.26     Uber 2018-11-26 03:30:00            South Station   \n",
       "330         1.40     Uber 2018-11-26 04:30:00        Boston University   \n",
       "30645       1.08     Uber 2018-11-26 04:30:00  Northeastern University   \n",
       "37300       1.10     Uber 2018-11-26 04:30:00  Northeastern University   \n",
       "58139       1.44     Uber 2018-11-26 04:30:00        Boston University   \n",
       "\n",
       "          source  price  surge_multiplier      id      name  temp  location  \\\n",
       "254787  Back Bay   34.0               1.0  279026     Black   NaN  Back Bay   \n",
       "330     Back Bay    4.5               1.0  203607  UberPool   NaN  Back Bay   \n",
       "30645   Back Bay    7.5               1.0  150814       WAV   NaN  Back Bay   \n",
       "37300   Back Bay    7.5               1.0  637167     UberX   NaN  Back Bay   \n",
       "58139   Back Bay    8.5               1.0   27464       WAV   NaN  Back Bay   \n",
       "\n",
       "        clouds  pressure  rain weather_time_stamp  humidity  wind  \n",
       "254787     NaN       NaN   NaN                NaT       NaN   NaN  \n",
       "330        NaN       NaN   NaN                NaT       NaN   NaN  \n",
       "30645      NaN       NaN   NaN                NaT       NaN   NaN  \n",
       "37300      NaN       NaN   NaN                NaT       NaN   NaN  \n",
       "58139      NaN       NaN   NaN                NaT       NaN   NaN  "
      ],
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>distance</th>\n",
       "      <th>cab_type</th>\n",
       "      <th>uber_time_stamp</th>\n",
       "      <th>destination</th>\n",
       "      <th>source</th>\n",
       "      <th>price</th>\n",
       "      <th>surge_multiplier</th>\n",
       "      <th>id</th>\n",
       "      <th>name</th>\n",
       "      <th>temp</th>\n",
       "      <th>location</th>\n",
       "      <th>clouds</th>\n",
       "      <th>pressure</th>\n",
       "      <th>rain</th>\n",
       "      <th>weather_time_stamp</th>\n",
       "      <th>humidity</th>\n",
       "      <th>wind</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>254787</th>\n",
       "      <td>6.26</td>\n",
       "      <td>Uber</td>\n",
       "      <td>2018-11-26 03:30:00</td>\n",
       "      <td>South Station</td>\n",
       "      <td>Back Bay</td>\n",
       "      <td>34.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>279026</td>\n",
       "      <td>Black</td>\n",
       "      <td>NaN</td>\n",
       "      <td>Back Bay</td>\n",
       "      <td>NaN</td>\n",
       "      <td>NaN</td>\n",
       "      <td>NaN</td>\n",
       "      <td>NaT</td>\n",
       "      <td>NaN</td>\n",
       "      <td>NaN</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>330</th>\n",
       "      <td>1.40</td>\n",
       "      <td>Uber</td>\n",
       "      <td>2018-11-26 04:30:00</td>\n",
       "      <td>Boston University</td>\n",
       "      <td>Back Bay</td>\n",
       "      <td>4.5</td>\n",
       "      <td>1.0</td>\n",
       "      <td>203607</td>\n",
       "      <td>UberPool</td>\n",
       "      <td>NaN</td>\n",
       "      <td>Back Bay</td>\n",
       "      <td>NaN</td>\n",
       "      <td>NaN</td>\n",
       "      <td>NaN</td>\n",
       "      <td>NaT</td>\n",
       "      <td>NaN</td>\n",
       "      <td>NaN</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>30645</th>\n",
       "      <td>1.08</td>\n",
       "      <td>Uber</td>\n",
       "      <td>2018-11-26 04:30:00</td>\n",
       "      <td>Northeastern University</td>\n",
       "      <td>Back Bay</td>\n",
       "      <td>7.5</td>\n",
       "      <td>1.0</td>\n",
       "      <td>150814</td>\n",
       "      <td>WAV</td>\n",
       "      <td>NaN</td>\n",
       "      <td>Back Bay</td>\n",
       "      <td>NaN</td>\n",
       "      <td>NaN</td>\n",
       "      <td>NaN</td>\n",
       "      <td>NaT</td>\n",
       "      <td>NaN</td>\n",
       "      <td>NaN</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>37300</th>\n",
       "      <td>1.10</td>\n",
       "      <td>Uber</td>\n",
       "      <td>2018-11-26 04:30:00</td>\n",
       "      <td>Northeastern University</td>\n",
       "      <td>Back Bay</td>\n",
       "      <td>7.5</td>\n",
       "      <td>1.0</td>\n",
       "      <td>637167</td>\n",
       "      <td>UberX</td>\n",
       "      <td>NaN</td>\n",
       "      <td>Back Bay</td>\n",
       "      <td>NaN</td>\n",
       "      <td>NaN</td>\n",
       "      <td>NaN</td>\n",
       "      <td>NaT</td>\n",
       "      <td>NaN</td>\n",
       "      <td>NaN</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>58139</th>\n",
       "      <td>1.44</td>\n",
       "      <td>Uber</td>\n",
       "      <td>2018-11-26 04:30:00</td>\n",
       "      <td>Boston University</td>\n",
       "      <td>Back Bay</td>\n",
       "      <td>8.5</td>\n",
       "      <td>1.0</td>\n",
       "      <td>27464</td>\n",
       "      <td>WAV</td>\n",
       "      <td>NaN</td>\n",
       "      <td>Back Bay</td>\n",
       "      <td>NaN</td>\n",
       "      <td>NaN</td>\n",
       "      <td>NaN</td>\n",
       "      <td>NaT</td>\n",
       "      <td>NaN</td>\n",
       "      <td>NaN</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ]
     },
     "execution_count": 12,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "execution_count": 12
  },
  {
   "cell_type": "code",
   "metadata": {
    "ExecuteTime": {
     "end_time": "2024-11-30T20:53:28.399542Z",
     "start_time": "2024-11-30T20:53:28.038358Z"
    }
   },
   "source": [
    "# Ensure the DataFrame is sorted by location and timestamp\n",
    "joined_df = joined_df.sort_values(by=['location', 'uber_time_stamp'])\n",
    "\n",
    "# List of weather columns to fill\n",
    "weather_columns = ['temp', 'clouds', 'pressure', 'rain', 'humidity', 'wind']\n",
    "\n",
    "# Create a helper column to track the source timestamp for weather data\n",
    "joined_df['weather_source_timestamp'] = joined_df['uber_time_stamp']\n",
    "\n",
    "# Function to fill weather columns and propagate the source timestamp\n",
    "def fill_weather_and_timestamp(group):\n",
    "    # Fill weather columns using nearest interpolation\n",
    "    for col in weather_columns:\n",
    "        group[col] = group[col].interpolate(method='nearest', limit_direction='both')\n",
    "    \n",
    "    # Propagate the source timestamp for missing weather values\n",
    "    group['weather_source_timestamp'] = group['weather_source_timestamp'].where(\n",
    "        group[weather_columns].notna().all(axis=1)  # Ensure all weather columns are non-NaN\n",
    "    ).ffill().bfill()  # Fill forward and backward for timestamps\n",
    "    \n",
    "    return group\n",
    "\n",
    "# Apply the function group-wise\n",
    "joined_df = joined_df.groupby('location').apply(fill_weather_and_timestamp)\n",
    "\n",
    "# Check the result\n",
    "joined_df.head(5)"
   ],
   "outputs": [
    {
     "data": {
      "text/plain": [
       "                 distance cab_type     uber_time_stamp  \\\n",
       "location                                                 \n",
       "Back Bay 254787      6.26     Uber 2018-11-26 03:30:00   \n",
       "         330         1.40     Uber 2018-11-26 04:30:00   \n",
       "         30645       1.08     Uber 2018-11-26 04:30:00   \n",
       "         37300       1.10     Uber 2018-11-26 04:30:00   \n",
       "         58139       1.44     Uber 2018-11-26 04:30:00   \n",
       "\n",
       "                             destination    source  price  surge_multiplier  \\\n",
       "location                                                                      \n",
       "Back Bay 254787            South Station  Back Bay   34.0               1.0   \n",
       "         330           Boston University  Back Bay    4.5               1.0   \n",
       "         30645   Northeastern University  Back Bay    7.5               1.0   \n",
       "         37300   Northeastern University  Back Bay    7.5               1.0   \n",
       "         58139         Boston University  Back Bay    8.5               1.0   \n",
       "\n",
       "                     id      name   temp  location  clouds  pressure    rain  \\\n",
       "location                                                                       \n",
       "Back Bay 254787  279026     Black  40.39  Back Bay    0.92   1013.79  0.0000   \n",
       "         330     203607  UberPool    NaN  Back Bay     NaN       NaN     NaN   \n",
       "         30645   150814       WAV  38.02  Back Bay    1.00   1020.06  0.0182   \n",
       "         37300   637167     UberX  36.42  Back Bay    0.90   1018.82  0.0000   \n",
       "         58139    27464       WAV  41.18  Back Bay    0.89   1026.16  0.0000   \n",
       "\n",
       "                weather_time_stamp  humidity  wind weather_source_timestamp  \n",
       "location                                                                     \n",
       "Back Bay 254787                NaT      0.93  2.71      2018-11-26 03:30:00  \n",
       "         330                   NaT       NaN   NaN      2018-11-26 03:30:00  \n",
       "         30645                 NaT      0.86  1.87      2018-11-26 04:30:00  \n",
       "         37300                 NaT      0.70  3.73      2018-11-26 04:30:00  \n",
       "         58139                 NaT      0.85  5.17      2018-11-26 04:30:00  "
      ],
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th>distance</th>\n",
       "      <th>cab_type</th>\n",
       "      <th>uber_time_stamp</th>\n",
       "      <th>destination</th>\n",
       "      <th>source</th>\n",
       "      <th>price</th>\n",
       "      <th>surge_multiplier</th>\n",
       "      <th>id</th>\n",
       "      <th>name</th>\n",
       "      <th>temp</th>\n",
       "      <th>location</th>\n",
       "      <th>clouds</th>\n",
       "      <th>pressure</th>\n",
       "      <th>rain</th>\n",
       "      <th>weather_time_stamp</th>\n",
       "      <th>humidity</th>\n",
       "      <th>wind</th>\n",
       "      <th>weather_source_timestamp</th>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>location</th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th rowspan=\"5\" valign=\"top\">Back Bay</th>\n",
       "      <th>254787</th>\n",
       "      <td>6.26</td>\n",
       "      <td>Uber</td>\n",
       "      <td>2018-11-26 03:30:00</td>\n",
       "      <td>South Station</td>\n",
       "      <td>Back Bay</td>\n",
       "      <td>34.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>279026</td>\n",
       "      <td>Black</td>\n",
       "      <td>40.39</td>\n",
       "      <td>Back Bay</td>\n",
       "      <td>0.92</td>\n",
       "      <td>1013.79</td>\n",
       "      <td>0.0000</td>\n",
       "      <td>NaT</td>\n",
       "      <td>0.93</td>\n",
       "      <td>2.71</td>\n",
       "      <td>2018-11-26 03:30:00</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>330</th>\n",
       "      <td>1.40</td>\n",
       "      <td>Uber</td>\n",
       "      <td>2018-11-26 04:30:00</td>\n",
       "      <td>Boston University</td>\n",
       "      <td>Back Bay</td>\n",
       "      <td>4.5</td>\n",
       "      <td>1.0</td>\n",
       "      <td>203607</td>\n",
       "      <td>UberPool</td>\n",
       "      <td>NaN</td>\n",
       "      <td>Back Bay</td>\n",
       "      <td>NaN</td>\n",
       "      <td>NaN</td>\n",
       "      <td>NaN</td>\n",
       "      <td>NaT</td>\n",
       "      <td>NaN</td>\n",
       "      <td>NaN</td>\n",
       "      <td>2018-11-26 03:30:00</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>30645</th>\n",
       "      <td>1.08</td>\n",
       "      <td>Uber</td>\n",
       "      <td>2018-11-26 04:30:00</td>\n",
       "      <td>Northeastern University</td>\n",
       "      <td>Back Bay</td>\n",
       "      <td>7.5</td>\n",
       "      <td>1.0</td>\n",
       "      <td>150814</td>\n",
       "      <td>WAV</td>\n",
       "      <td>38.02</td>\n",
       "      <td>Back Bay</td>\n",
       "      <td>1.00</td>\n",
       "      <td>1020.06</td>\n",
       "      <td>0.0182</td>\n",
       "      <td>NaT</td>\n",
       "      <td>0.86</td>\n",
       "      <td>1.87</td>\n",
       "      <td>2018-11-26 04:30:00</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>37300</th>\n",
       "      <td>1.10</td>\n",
       "      <td>Uber</td>\n",
       "      <td>2018-11-26 04:30:00</td>\n",
       "      <td>Northeastern University</td>\n",
       "      <td>Back Bay</td>\n",
       "      <td>7.5</td>\n",
       "      <td>1.0</td>\n",
       "      <td>637167</td>\n",
       "      <td>UberX</td>\n",
       "      <td>36.42</td>\n",
       "      <td>Back Bay</td>\n",
       "      <td>0.90</td>\n",
       "      <td>1018.82</td>\n",
       "      <td>0.0000</td>\n",
       "      <td>NaT</td>\n",
       "      <td>0.70</td>\n",
       "      <td>3.73</td>\n",
       "      <td>2018-11-26 04:30:00</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>58139</th>\n",
       "      <td>1.44</td>\n",
       "      <td>Uber</td>\n",
       "      <td>2018-11-26 04:30:00</td>\n",
       "      <td>Boston University</td>\n",
       "      <td>Back Bay</td>\n",
       "      <td>8.5</td>\n",
       "      <td>1.0</td>\n",
       "      <td>27464</td>\n",
       "      <td>WAV</td>\n",
       "      <td>41.18</td>\n",
       "      <td>Back Bay</td>\n",
       "      <td>0.89</td>\n",
       "      <td>1026.16</td>\n",
       "      <td>0.0000</td>\n",
       "      <td>NaT</td>\n",
       "      <td>0.85</td>\n",
       "      <td>5.17</td>\n",
       "      <td>2018-11-26 04:30:00</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ]
     },
     "execution_count": 13,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "execution_count": 13
  },
  {
   "cell_type": "code",
   "metadata": {
    "ExecuteTime": {
     "end_time": "2024-11-30T20:53:30.743091Z",
     "start_time": "2024-11-30T20:53:30.656143Z"
    }
   },
   "source": [
    "# Step 1: Replace 'weather_time_stamp' with 'weather_source_timestamp' where available\n",
    "joined_df['weather_time_stamp'] = joined_df['weather_source_timestamp'].combine_first(joined_df['weather_time_stamp'])\n",
    "\n",
    "# Step 2: Remove 'weather_source_timestamp' column\n",
    "joined_df = joined_df.drop(columns=['weather_source_timestamp'])\n",
    "\n",
    "# Step 3: Count rows with any null value\n",
    "rows_before = len(joined_df)\n",
    "joined_df = joined_df.dropna()  # Remove rows with any null values\n",
    "rows_after = len(joined_df)\n",
    "\n",
    "# Step 4: Print the number of rows removed\n",
    "rows_removed = rows_before - rows_after\n",
    "print(f\"Number of rows removed: {rows_removed}\")"
   ],
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Number of rows removed: 17\n"
     ]
    }
   ],
   "execution_count": 14
  },
  {
   "cell_type": "code",
   "metadata": {
    "ExecuteTime": {
     "end_time": "2024-11-30T20:53:32.288224Z",
     "start_time": "2024-11-30T20:53:32.240226Z"
    }
   },
   "source": [
    "from sklearn.preprocessing import StandardScaler\n",
    "\n",
    "# Select the weather-related columns and price\n",
    "weather_columns = ['temp', 'clouds', 'pressure', 'rain', 'humidity', 'wind', 'price']\n",
    "selected_df = joined_df[weather_columns]\n",
    "\n",
    "# Separate the weather columns and price\n",
    "weather_data = selected_df.drop(columns=['price'])\n",
    "price_data = selected_df['price']\n",
    "\n",
    "# Initialize the StandardScaler\n",
    "scaler = StandardScaler()\n",
    "\n",
    "# Standardize the weather data\n",
    "scaled_data = scaler.fit_transform(weather_data)\n",
    "\n",
    "# Create a DataFrame with the scaled values\n",
    "scaled_df = pd.DataFrame(scaled_data, columns=weather_data.columns, index=selected_df.index)\n",
    "\n",
    "# Add the 'price' column back (ensure index is aligned)\n",
    "scaled_df['price'] = price_data\n",
    "\n",
    "# Calculate the correlation\n",
    "correlation_with_price_scaled = scaled_df.corr()['price'].sort_values(ascending=False)\n",
    "\n",
    "# Print the correlations\n",
    "print(\"Correlation with price (scaled weather columns):\")\n",
    "print(correlation_with_price_scaled)\n"
   ],
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Correlation with price (scaled weather columns):\n",
      "price       1.000000\n",
      "temp        0.004259\n",
      "clouds      0.003939\n",
      "humidity    0.003918\n",
      "pressure    0.001107\n",
      "rain       -0.005391\n",
      "wind       -0.006198\n",
      "Name: price, dtype: float64\n"
     ]
    }
   ],
   "execution_count": 15
  },
  {
   "cell_type": "code",
   "metadata": {
    "ExecuteTime": {
     "end_time": "2024-11-30T20:53:36.402123Z",
     "start_time": "2024-11-30T20:53:34.544095Z"
    }
   },
   "source": [
    "import seaborn as sns\n",
    "import matplotlib.pyplot as plt\n",
    "\n",
    "# Plot scatter plots for weather features against price\n",
    "weather_columns = ['temp', 'clouds', 'humidity', 'pressure', 'rain', 'wind']\n",
    "\n",
    "for col in weather_columns:\n",
    "    plt.figure(figsize=(8, 6))\n",
    "    sns.scatterplot(x=joined_df[col], y=joined_df['price'])\n",
    "    plt.title(f'{col} vs Price')\n",
    "    plt.xlabel(col)\n",
    "    plt.ylabel('Price')\n",
    "    plt.show()"
   ],
   "outputs": [
    {
     "data": {
      "text/plain": [
       "<Figure size 800x600 with 1 Axes>"
      ],
      "image/png": "..."
     },
     "metadata": {},
     "output_type": "display_data"
    },
    {
     "data": {
      "text/plain": [
       "<Figure size 800x600 with 1 Axes>"
      ],
      "image/png": "..."
     },
     "metadata": {},
     "output_type": "display_data"
    },
    {
     "data": {
      "text/plain": [
       "<Figure size 800x600 with 1 Axes>"
      ],
      "image/png": "..."
     },
     "metadata": {},
     "output_type": "display_data"
    },
    {
     "data": {
      "text/plain": [
       "<Figure size 800x600 with 1 Axes>"
      ],
      "image/png": "..."
     },
     "metadata": {},
     "output_type": "display_data"
    },
    {
     "data": {
      "text/plain": [
       "<Figure size 800x600 with 1 Axes>"
      ],
      "image/png": "..."
     },
     "metadata": {},
     "output_type": "display_data"
    },
    {
     "data": {
      "text/plain": [
       "<Figure size 800x600 with 1 Axes>"
      ],
      "image/png": "..."
     },
     "metadata": {},
     "output_type": "display_data"
    }
   ],
   "execution_count": 16
  },
  {
   "cell_type": "code",
   "metadata": {
    "ExecuteTime": {
     "end_time": "2024-12-01T19:59:14.162549Z",
     "start_time": "2024-12-01T19:59:14.113666Z"
    }
   },
   "source": [
    "# importing libraries for machine learning\n",
    "\n",
    "import pandas as pd\n",
    "from sklearn.ensemble import RandomForestRegressor\n",
    "from sklearn.model_selection import train_test_split\n",
    "from sklearn.compose import ColumnTransformer\n",
    "from sklearn.preprocessing import OneHotEncoder\n",
    "from sklearn.pipeline import Pipeline\n",
    "from sklearn.metrics import accuracy_score, classification_report, r2_score\n",
    "\n",
    "\n",
    "# change features depending on correlation metrics/what will make model most accurate\n",
    "features = ['distance', 'cab_type', 'uber_time_stamp', 'destination', 'source', 'surge_multiplier']\n",
    "\n",
    "# variable that is being predicted\n",
    "target_variable = 'price'\n",
    "\n",
    "# Extract X (features) and y (target) from the data\n",
    "X = joined_df[features]\n",
    "y = joined_df[target_variable]\n"
   ],
   "outputs": [],
   "execution_count": 23
  },
  {
   "cell_type": "code",
   "metadata": {
    "ExecuteTime": {
     "end_time": "2024-12-01T19:59:31.335154Z",
     "start_time": "2024-12-01T19:59:31.298753Z"
    }
   },
   "source": [
    "# preprocessing for model\n",
    "\n",
    "# Convert 'time_stamp' into datetime format\n",
    "X['uber_time_stamp'] = pd.to_datetime(X['uber_time_stamp'])\n",
    "\n",
    "# Extract useful time-based features\n",
    "X['hour'] = X['uber_time_stamp'].dt.hour\n",
    "X['day_of_week'] = X['uber_time_stamp'].dt.dayofweek\n",
    "\n",
    "# Drop the original 'time_stamp' column as it's no longer needed\n",
    "X = X.drop('uber_time_stamp', axis=1)"
   ],
   "outputs": [],
   "execution_count": 25
  },
  {
   "cell_type": "code",
   "metadata": {
    "ExecuteTime": {
     "end_time": "2024-12-01T19:59:34.604411Z",
     "start_time": "2024-12-01T19:59:34.601115Z"
    }
   },
   "source": [
    "# Configures a ColumnTransformer to preprocess categorical features using OneHotEncoder\n",
    "# leaves numerical features unchanged for model training\n",
    "\n",
    "\n",
    "# Define categorical and numerical features\n",
    "categorical_features = ['cab_type', 'source', 'destination']\n",
    "numerical_features = ['distance', 'surge_multiplier', 'hour', 'day_of_week']\n",
    "\n",
    "# Set up the ColumnTransformer\n",
    "preprocessor = ColumnTransformer(\n",
    "    transformers=[\n",
    "        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),\n",
    "        ('num', 'passthrough', numerical_features)\n",
    "    ]\n",
    ")\n",
    "\n",
    "# Display the preprocessor to confirm setup\n",
    "print(preprocessor)"
   ],
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "ColumnTransformer(transformers=[('cat', OneHotEncoder(handle_unknown='ignore'),\n",
      "                                 ['cab_type', 'source', 'destination']),\n",
      "                                ('num', 'passthrough',\n",
      "                                 ['distance', 'surge_multiplier', 'hour',\n",
      "                                  'day_of_week'])])\n"
     ]
    }
   ],
   "execution_count": 26
  },
  {
   "cell_type": "code",
   "metadata": {
    "ExecuteTime": {
     "end_time": "2024-12-01T19:59:37.201571Z",
     "start_time": "2024-12-01T19:59:37.168058Z"
    }
   },
   "source": [
    "# splitting data into training and testing sets\n",
    "\n",
    "X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)"
   ],
   "outputs": [],
   "execution_count": 27
  },
  {
   "cell_type": "code",
   "metadata": {
    "ExecuteTime": {
     "end_time": "2024-12-01T20:06:40.976728Z",
     "start_time": "2024-12-01T20:00:51.649683Z"
    }
   },
   "source": [
    "# creating pipeline and training the model\n",
    "\n",
    "# Set up the Random Forest model\n",
    "rf_model = RandomForestRegressor(n_estimators=100, random_state=42)\n",
    "\n",
    "# Create a pipeline with preprocessing and the model\n",
    "pipeline = Pipeline(steps=[\n",
    "    ('preprocessor', preprocessor),\n",
    "    ('regressor', rf_model)\n",
    "])\n",
    "\n",
    "# Train the pipeline on the training data\n",
    "pipeline.fit(X_train, y_train)\n",
    "\n",
    "print(\"Pipeline training complete.\")"
   ],
   "outputs": [
    {
     "ename": "KeyboardInterrupt",
     "evalue": "",
     "output_type": "error",
     "traceback": [
      "\u001B[0;31m---------------------------------------------------------------------------\u001B[0m",
      "\u001B[0;31mKeyboardInterrupt\u001B[0m                         Traceback (most recent call last)",
      "Cell \u001B[0;32mIn[29], line 13\u001B[0m\n\u001B[1;32m      7\u001B[0m pipeline \u001B[38;5;241m=\u001B[39m Pipeline(steps\u001B[38;5;241m=\u001B[39m[\n\u001B[1;32m      8\u001B[0m     (\u001B[38;5;124m'\u001B[39m\u001B[38;5;124mpreprocessor\u001B[39m\u001B[38;5;124m'\u001B[39m, preprocessor),\n\u001B[1;32m      9\u001B[0m     (\u001B[38;5;124m'\u001B[39m\u001B[38;5;124mregressor\u001B[39m\u001B[38;5;124m'\u001B[39m, rf_model)\n\u001B[1;32m     10\u001B[0m ])\n\u001B[1;32m     12\u001B[0m \u001B[38;5;66;03m# Train the pipeline on the training data\u001B[39;00m\n\u001B[0;32m---> 13\u001B[0m pipeline\u001B[38;5;241m.\u001B[39mfit(X_train, y_train)\n\u001B[1;32m     15\u001B[0m \u001B[38;5;28mprint\u001B[39m(\u001B[38;5;124m\"\u001B[39m\u001B[38;5;124mPipeline training complete.\u001B[39m\u001B[38;5;124m\"\u001B[39m)\n",
      "File \u001B[0;32m~/anaconda3/envs/DS3500/lib/python3.12/site-packages/sklearn/base.py:1473\u001B[0m, in \u001B[0;36m_fit_context.<locals>.decorator.<locals>.wrapper\u001B[0;34m(estimator, *args, **kwargs)\u001B[0m\n\u001B[1;32m   1466\u001B[0m     estimator\u001B[38;5;241m.\u001B[39m_validate_params()\n\u001B[1;32m   1468\u001B[0m \u001B[38;5;28;01mwith\u001B[39;00m config_context(\n\u001B[1;32m   1469\u001B[0m     skip_parameter_validation\u001B[38;5;241m=\u001B[39m(\n\u001B[1;32m   1470\u001B[0m         prefer_skip_nested_validation \u001B[38;5;129;01mor\u001B[39;00m global_skip_validation\n\u001B[1;32m   1471\u001B[0m     )\n\u001B[1;32m   1472\u001B[0m ):\n\u001B[0;32m-> 1473\u001B[0m     \u001B[38;5;28;01mreturn\u001B[39;00m fit_method(estimator, \u001B[38;5;241m*\u001B[39margs, \u001B[38;5;241m*\u001B[39m\u001B[38;5;241m*\u001B[39mkwargs)\n",
      "File \u001B[0;32m~/anaconda3/envs/DS3500/lib/python3.12/site-packages/sklearn/pipeline.py:473\u001B[0m, in \u001B[0;36mPipeline.fit\u001B[0;34m(self, X, y, **params)\u001B[0m\n\u001B[1;32m    471\u001B[0m     \u001B[38;5;28;01mif\u001B[39;00m \u001B[38;5;28mself\u001B[39m\u001B[38;5;241m.\u001B[39m_final_estimator \u001B[38;5;241m!=\u001B[39m \u001B[38;5;124m\"\u001B[39m\u001B[38;5;124mpassthrough\u001B[39m\u001B[38;5;124m\"\u001B[39m:\n\u001B[1;32m    472\u001B[0m         last_step_params \u001B[38;5;241m=\u001B[39m routed_params[\u001B[38;5;28mself\u001B[39m\u001B[38;5;241m.\u001B[39msteps[\u001B[38;5;241m-\u001B[39m\u001B[38;5;241m1\u001B[39m][\u001B[38;5;241m0\u001B[39m]]\n\u001B[0;32m--> 473\u001B[0m         \u001B[38;5;28mself\u001B[39m\u001B[38;5;241m.\u001B[39m_final_estimator\u001B[38;5;241m.\u001B[39mfit(Xt, y, \u001B[38;5;241m*\u001B[39m\u001B[38;5;241m*\u001B[39mlast_step_params[\u001B[38;5;124m\"\u001B[39m\u001B[38;5;124mfit\u001B[39m\u001B[38;5;124m\"\u001B[39m])\n\u001B[1;32m    475\u001B[0m \u001B[38;5;28;01mreturn\u001B[39;00m \u001B[38;5;28mself\u001B[39m\n",
      "File \u001B[0;32m~/anaconda3/envs/DS3500/lib/python3.12/site-packages/sklearn/base.py:1473\u001B[0m, in \u001B[0;36m_fit_context.<locals>.decorator.<locals>.wrapper\u001B[0;34m(estimator, *args, **kwargs)\u001B[0m\n\u001B[1;32m   1466\u001B[0m     estimator\u001B[38;5;241m.\u001B[39m_validate_params()\n\u001B[1;32m   1468\u001B[0m \u001B[38;5;28;01mwith\u001B[39;00m config_context(\n\u001B[1;32m   1469\u001B[0m     skip_parameter_validation\u001B[38;5;241m=\u001B[39m(\n\u001B[1;32m   1470\u001B[0m         prefer_skip_nested_validation \u001B[38;5;129;01mor\u001B[39;00m global_skip_validation\n\u001B[1;32m   1471\u001B[0m     )\n\u001B[1;32m   1472\u001B[0m ):\n\u001B[0;32m-> 1473\u001B[0m     \u001B[38;5;28;01mreturn\u001B[39;00m fit_method(estimator, \u001B[38;5;241m*\u001B[39margs, \u001B[38;5;241m*\u001B[39m\u001B[38;5;241m*\u001B[39mkwargs)\n",
      "File \u001B[0;32m~/anaconda3/envs/DS3500/lib/python3.12/site-packages/sklearn/ensemble/_forest.py:489\u001B[0m, in \u001B[0;36mBaseForest.fit\u001B[0;34m(self, X, y, sample_weight)\u001B[0m\n\u001B[1;32m    478\u001B[0m trees \u001B[38;5;241m=\u001B[39m [\n\u001B[1;32m    479\u001B[0m     \u001B[38;5;28mself\u001B[39m\u001B[38;5;241m.\u001B[39m_make_estimator(append\u001B[38;5;241m=\u001B[39m\u001B[38;5;28;01mFalse\u001B[39;00m, random_state\u001B[38;5;241m=\u001B[39mrandom_state)\n\u001B[1;32m    480\u001B[0m     \u001B[38;5;28;01mfor\u001B[39;00m i \u001B[38;5;129;01min\u001B[39;00m \u001B[38;5;28mrange\u001B[39m(n_more_estimators)\n\u001B[1;32m    481\u001B[0m ]\n\u001B[1;32m    483\u001B[0m \u001B[38;5;66;03m# Parallel loop: we prefer the threading backend as the Cython code\u001B[39;00m\n\u001B[1;32m    484\u001B[0m \u001B[38;5;66;03m# for fitting the trees is internally releasing the Python GIL\u001B[39;00m\n\u001B[1;32m    485\u001B[0m \u001B[38;5;66;03m# making threading more efficient than multiprocessing in\u001B[39;00m\n\u001B[1;32m    486\u001B[0m \u001B[38;5;66;03m# that case. However, for joblib 0.12+ we respect any\u001B[39;00m\n\u001B[1;32m    487\u001B[0m \u001B[38;5;66;03m# parallel_backend contexts set at a higher level,\u001B[39;00m\n\u001B[1;32m    488\u001B[0m \u001B[38;5;66;03m# since correctness does not rely on using threads.\u001B[39;00m\n\u001B[0;32m--> 489\u001B[0m trees \u001B[38;5;241m=\u001B[39m Parallel(\n\u001B[1;32m    490\u001B[0m     n_jobs\u001B[38;5;241m=\u001B[39m\u001B[38;5;28mself\u001B[39m\u001B[38;5;241m.\u001B[39mn_jobs,\n\u001B[1;32m    491\u001B[0m     verbose\u001B[38;5;241m=\u001B[39m\u001B[38;5;28mself\u001B[39m\u001B[38;5;241m.\u001B[39mverbose,\n\u001B[1;32m    492\u001B[0m     prefer\u001B[38;5;241m=\u001B[39m\u001B[38;5;124m\"\u001B[39m\u001B[38;5;124mthreads\u001B[39m\u001B[38;5;124m\"\u001B[39m,\n\u001B[1;32m    493\u001B[0m )(\n\u001B[1;32m    494\u001B[0m     delayed(_parallel_build_trees)(\n\u001B[1;32m    495\u001B[0m         t,\n\u001B[1;32m    496\u001B[0m         \u001B[38;5;28mself\u001B[39m\u001B[38;5;241m.\u001B[39mbootstrap,\n\u001B[1;32m    497\u001B[0m         X,\n\u001B[1;32m    498\u001B[0m         y,\n\u001B[1;32m    499\u001B[0m         sample_weight,\n\u001B[1;32m    500\u001B[0m         i,\n\u001B[1;32m    501\u001B[0m         \u001B[38;5;28mlen\u001B[39m(trees),\n\u001B[1;32m    502\u001B[0m         verbose\u001B[38;5;241m=\u001B[39m\u001B[38;5;28mself\u001B[39m\u001B[38;5;241m.\u001B[39mverbose,\n\u001B[1;32m    503\u001B[0m         class_weight\u001B[38;5;241m=\u001B[39m\u001B[38;5;28mself\u001B[39m\u001B[38;5;241m.\u001B[39mclass_weight,\n\u001B[1;32m    504\u001B[0m         n_samples_bootstrap\u001B[38;5;241m=\u001B[39mn_samples_bootstrap,\n\u001B[1;32m    505\u001B[0m         missing_values_in_feature_mask\u001B[38;5;241m=\u001B[39mmissing_values_in_feature_mask,\n\u001B[1;32m    506\u001B[0m     )\n\u001B[1;32m    507\u001B[0m     \u001B[38;5;28;01mfor\u001B[39;00m i, t \u001B[38;5;129;01min\u001B[39;00m \u001B[38;5;28menumerate\u001B[39m(trees)\n\u001B[1;32m    508\u001B[0m )\n\u001B[1;32m    510\u001B[0m \u001B[38;5;66;03m# Collect newly grown trees\u001B[39;00m\n\u001B[1;32m    511\u001B[0m \u001B[38;5;28mself\u001B[39m\u001B[38;5;241m.\u001B[39mestimators_\u001B[38;5;241m.\u001B[39mextend(trees)\n",
      "File \u001B[0;32m~/anaconda3/envs/DS3500/lib/python3.12/site-packages/sklearn/utils/parallel.py:74\u001B[0m, in \u001B[0;36mParallel.__call__\u001B[0;34m(self, iterable)\u001B[0m\n\u001B[1;32m     69\u001B[0m config \u001B[38;5;241m=\u001B[39m get_config()\n\u001B[1;32m     70\u001B[0m iterable_with_config \u001B[38;5;241m=\u001B[39m (\n\u001B[1;32m     71\u001B[0m     (_with_config(delayed_func, config), args, kwargs)\n\u001B[1;32m     72\u001B[0m     \u001B[38;5;28;01mfor\u001B[39;00m delayed_func, args, kwargs \u001B[38;5;129;01min\u001B[39;00m iterable\n\u001B[1;32m     73\u001B[0m )\n\u001B[0;32m---> 74\u001B[0m \u001B[38;5;28;01mreturn\u001B[39;00m \u001B[38;5;28msuper\u001B[39m()\u001B[38;5;241m.\u001B[39m\u001B[38;5;21m__call__\u001B[39m(iterable_with_config)\n",
      "File \u001B[0;32m~/anaconda3/envs/DS3500/lib/python3.12/site-packages/joblib/parallel.py:1918\u001B[0m, in \u001B[0;36mParallel.__call__\u001B[0;34m(self, iterable)\u001B[0m\n\u001B[1;32m   1916\u001B[0m     output \u001B[38;5;241m=\u001B[39m \u001B[38;5;28mself\u001B[39m\u001B[38;5;241m.\u001B[39m_get_sequential_output(iterable)\n\u001B[1;32m   1917\u001B[0m     \u001B[38;5;28mnext\u001B[39m(output)\n\u001B[0;32m-> 1918\u001B[0m     \u001B[38;5;28;01mreturn\u001B[39;00m output \u001B[38;5;28;01mif\u001B[39;00m \u001B[38;5;28mself\u001B[39m\u001B[38;5;241m.\u001B[39mreturn_generator \u001B[38;5;28;01melse\u001B[39;00m \u001B[38;5;28mlist\u001B[39m(output)\n\u001B[1;32m   1920\u001B[0m \u001B[38;5;66;03m# Let's create an ID that uniquely identifies the current call. If the\u001B[39;00m\n\u001B[1;32m   1921\u001B[0m \u001B[38;5;66;03m# call is interrupted early and that the same instance is immediately\u001B[39;00m\n\u001B[1;32m   1922\u001B[0m \u001B[38;5;66;03m# re-used, this id will be used to prevent workers that were\u001B[39;00m\n\u001B[1;32m   1923\u001B[0m \u001B[38;5;66;03m# concurrently finalizing a task from the previous call to run the\u001B[39;00m\n\u001B[1;32m   1924\u001B[0m \u001B[38;5;66;03m# callback.\u001B[39;00m\n\u001B[1;32m   1925\u001B[0m \u001B[38;5;28;01mwith\u001B[39;00m \u001B[38;5;28mself\u001B[39m\u001B[38;5;241m.\u001B[39m_lock:\n",
      "File \u001B[0;32m~/anaconda3/envs/DS3500/lib/python3.12/site-packages/joblib/parallel.py:1847\u001B[0m, in \u001B[0;36mParallel._get_sequential_output\u001B[0;34m(self, iterable)\u001B[0m\n\u001B[1;32m   1845\u001B[0m \u001B[38;5;28mself\u001B[39m\u001B[38;5;241m.\u001B[39mn_dispatched_batches \u001B[38;5;241m+\u001B[39m\u001B[38;5;241m=\u001B[39m \u001B[38;5;241m1\u001B[39m\n\u001B[1;32m   1846\u001B[0m \u001B[38;5;28mself\u001B[39m\u001B[38;5;241m.\u001B[39mn_dispatched_tasks \u001B[38;5;241m+\u001B[39m\u001B[38;5;241m=\u001B[39m \u001B[38;5;241m1\u001B[39m\n\u001B[0;32m-> 1847\u001B[0m res \u001B[38;5;241m=\u001B[39m func(\u001B[38;5;241m*\u001B[39margs, \u001B[38;5;241m*\u001B[39m\u001B[38;5;241m*\u001B[39mkwargs)\n\u001B[1;32m   1848\u001B[0m \u001B[38;5;28mself\u001B[39m\u001B[38;5;241m.\u001B[39mn_completed_tasks \u001B[38;5;241m+\u001B[39m\u001B[38;5;241m=\u001B[39m \u001B[38;5;241m1\u001B[39m\n\u001B[1;32m   1849\u001B[0m \u001B[38;5;28mself\u001B[39m\u001B[38;5;241m.\u001B[39mprint_progress()\n",
      "File \u001B[0;32m~/anaconda3/envs/DS3500/lib/python3.12/site-packages/sklearn/utils/parallel.py:136\u001B[0m, in \u001B[0;36m_FuncWrapper.__call__\u001B[0;34m(self, *args, **kwargs)\u001B[0m\n\u001B[1;32m    134\u001B[0m     config \u001B[38;5;241m=\u001B[39m {}\n\u001B[1;32m    135\u001B[0m \u001B[38;5;28;01mwith\u001B[39;00m config_context(\u001B[38;5;241m*\u001B[39m\u001B[38;5;241m*\u001B[39mconfig):\n\u001B[0;32m--> 136\u001B[0m     \u001B[38;5;28;01mreturn\u001B[39;00m \u001B[38;5;28mself\u001B[39m\u001B[38;5;241m.\u001B[39mfunction(\u001B[38;5;241m*\u001B[39margs, \u001B[38;5;241m*\u001B[39m\u001B[38;5;241m*\u001B[39mkwargs)\n",
      "File \u001B[0;32m~/anaconda3/envs/DS3500/lib/python3.12/site-packages/sklearn/ensemble/_forest.py:192\u001B[0m, in \u001B[0;36m_parallel_build_trees\u001B[0;34m(tree, bootstrap, X, y, sample_weight, tree_idx, n_trees, verbose, class_weight, n_samples_bootstrap, missing_values_in_feature_mask)\u001B[0m\n\u001B[1;32m    189\u001B[0m     \u001B[38;5;28;01melif\u001B[39;00m class_weight \u001B[38;5;241m==\u001B[39m \u001B[38;5;124m\"\u001B[39m\u001B[38;5;124mbalanced_subsample\u001B[39m\u001B[38;5;124m\"\u001B[39m:\n\u001B[1;32m    190\u001B[0m         curr_sample_weight \u001B[38;5;241m*\u001B[39m\u001B[38;5;241m=\u001B[39m compute_sample_weight(\u001B[38;5;124m\"\u001B[39m\u001B[38;5;124mbalanced\u001B[39m\u001B[38;5;124m\"\u001B[39m, y, indices\u001B[38;5;241m=\u001B[39mindices)\n\u001B[0;32m--> 192\u001B[0m     tree\u001B[38;5;241m.\u001B[39m_fit(\n\u001B[1;32m    193\u001B[0m         X,\n\u001B[1;32m    194\u001B[0m         y,\n\u001B[1;32m    195\u001B[0m         sample_weight\u001B[38;5;241m=\u001B[39mcurr_sample_weight,\n\u001B[1;32m    196\u001B[0m         check_input\u001B[38;5;241m=\u001B[39m\u001B[38;5;28;01mFalse\u001B[39;00m,\n\u001B[1;32m    197\u001B[0m         missing_values_in_feature_mask\u001B[38;5;241m=\u001B[39mmissing_values_in_feature_mask,\n\u001B[1;32m    198\u001B[0m     )\n\u001B[1;32m    199\u001B[0m \u001B[38;5;28;01melse\u001B[39;00m:\n\u001B[1;32m    200\u001B[0m     tree\u001B[38;5;241m.\u001B[39m_fit(\n\u001B[1;32m    201\u001B[0m         X,\n\u001B[1;32m    202\u001B[0m         y,\n\u001B[0;32m   (...)\u001B[0m\n\u001B[1;32m    205\u001B[0m         missing_values_in_feature_mask\u001B[38;5;241m=\u001B[39mmissing_values_in_feature_mask,\n\u001B[1;32m    206\u001B[0m     )\n",
      "File \u001B[0;32m~/anaconda3/envs/DS3500/lib/python3.12/site-packages/sklearn/tree/_classes.py:472\u001B[0m, in \u001B[0;36mBaseDecisionTree._fit\u001B[0;34m(self, X, y, sample_weight, check_input, missing_values_in_feature_mask)\u001B[0m\n\u001B[1;32m    461\u001B[0m \u001B[38;5;28;01melse\u001B[39;00m:\n\u001B[1;32m    462\u001B[0m     builder \u001B[38;5;241m=\u001B[39m BestFirstTreeBuilder(\n\u001B[1;32m    463\u001B[0m         splitter,\n\u001B[1;32m    464\u001B[0m         min_samples_split,\n\u001B[0;32m   (...)\u001B[0m\n\u001B[1;32m    469\u001B[0m         \u001B[38;5;28mself\u001B[39m\u001B[38;5;241m.\u001B[39mmin_impurity_decrease,\n\u001B[1;32m    470\u001B[0m     )\n\u001B[0;32m--> 472\u001B[0m builder\u001B[38;5;241m.\u001B[39mbuild(\u001B[38;5;28mself\u001B[39m\u001B[38;5;241m.\u001B[39mtree_, X, y, sample_weight, missing_values_in_feature_mask)\n\u001B[1;32m    474\u001B[0m \u001B[38;5;28;01mif\u001B[39;00m \u001B[38;5;28mself\u001B[39m\u001B[38;5;241m.\u001B[39mn_outputs_ \u001B[38;5;241m==\u001B[39m \u001B[38;5;241m1\u001B[39m \u001B[38;5;129;01mand\u001B[39;00m is_classifier(\u001B[38;5;28mself\u001B[39m):\n\u001B[1;32m    475\u001B[0m     \u001B[38;5;28mself\u001B[39m\u001B[38;5;241m.\u001B[39mn_classes_ \u001B[38;5;241m=\u001B[39m \u001B[38;5;28mself\u001B[39m\u001B[38;5;241m.\u001B[39mn_classes_[\u001B[38;5;241m0\u001B[39m]\n",
      "\u001B[0;31mKeyboardInterrupt\u001B[0m: "
     ]
    }
   ],
   "execution_count": 29
  },
  {
   "cell_type": "code",
   "metadata": {
    "ExecuteTime": {
     "end_time": "2024-12-01T20:07:04.173749Z",
     "start_time": "2024-12-01T20:07:04.086791Z"
    }
   },
   "source": [
    "# making predictions and evaluating model\n",
    "\n",
    "# Make predictions on the test set\n",
    "y_pred = pipeline.predict(X_test)\n",
    "\n",
    "# Evaluate the model using MAE\n",
    "mae = mean_absolute_error(y_test, y_pred)\n",
    "print(f\"Mean Absolute Error (MAE): {mae}\")\n",
    "\n",
    "# Evaluate the model using R² Score\n",
    "r2 = r2_score(y_test, y_pred)\n",
    "print(f\"R² Score: {r2}\")"
   ],
   "outputs": [
    {
     "ename": "IndexError",
     "evalue": "list index out of range",
     "output_type": "error",
     "traceback": [
      "\u001B[0;31m---------------------------------------------------------------------------\u001B[0m",
      "\u001B[0;31mIndexError\u001B[0m                                Traceback (most recent call last)",
      "Cell \u001B[0;32mIn[30], line 4\u001B[0m\n\u001B[1;32m      1\u001B[0m \u001B[38;5;66;03m# making predictions and evaluating model\u001B[39;00m\n\u001B[1;32m      2\u001B[0m \n\u001B[1;32m      3\u001B[0m \u001B[38;5;66;03m# Make predictions on the test set\u001B[39;00m\n\u001B[0;32m----> 4\u001B[0m y_pred \u001B[38;5;241m=\u001B[39m pipeline\u001B[38;5;241m.\u001B[39mpredict(X_test)\n\u001B[1;32m      6\u001B[0m \u001B[38;5;66;03m# Evaluate the model using MAE\u001B[39;00m\n\u001B[1;32m      7\u001B[0m mae \u001B[38;5;241m=\u001B[39m mean_absolute_error(y_test, y_pred)\n",
      "File \u001B[0;32m~/anaconda3/envs/DS3500/lib/python3.12/site-packages/sklearn/pipeline.py:601\u001B[0m, in \u001B[0;36mPipeline.predict\u001B[0;34m(self, X, **params)\u001B[0m\n\u001B[1;32m    599\u001B[0m     \u001B[38;5;28;01mfor\u001B[39;00m _, name, transform \u001B[38;5;129;01min\u001B[39;00m \u001B[38;5;28mself\u001B[39m\u001B[38;5;241m.\u001B[39m_iter(with_final\u001B[38;5;241m=\u001B[39m\u001B[38;5;28;01mFalse\u001B[39;00m):\n\u001B[1;32m    600\u001B[0m         Xt \u001B[38;5;241m=\u001B[39m transform\u001B[38;5;241m.\u001B[39mtransform(Xt)\n\u001B[0;32m--> 601\u001B[0m     \u001B[38;5;28;01mreturn\u001B[39;00m \u001B[38;5;28mself\u001B[39m\u001B[38;5;241m.\u001B[39msteps[\u001B[38;5;241m-\u001B[39m\u001B[38;5;241m1\u001B[39m][\u001B[38;5;241m1\u001B[39m]\u001B[38;5;241m.\u001B[39mpredict(Xt, \u001B[38;5;241m*\u001B[39m\u001B[38;5;241m*\u001B[39mparams)\n\u001B[1;32m    603\u001B[0m \u001B[38;5;66;03m# metadata routing enabled\u001B[39;00m\n\u001B[1;32m    604\u001B[0m routed_params \u001B[38;5;241m=\u001B[39m process_routing(\u001B[38;5;28mself\u001B[39m, \u001B[38;5;124m\"\u001B[39m\u001B[38;5;124mpredict\u001B[39m\u001B[38;5;124m\"\u001B[39m, \u001B[38;5;241m*\u001B[39m\u001B[38;5;241m*\u001B[39mparams)\n",
      "File \u001B[0;32m~/anaconda3/envs/DS3500/lib/python3.12/site-packages/sklearn/ensemble/_forest.py:1063\u001B[0m, in \u001B[0;36mForestRegressor.predict\u001B[0;34m(self, X)\u001B[0m\n\u001B[1;32m   1061\u001B[0m check_is_fitted(\u001B[38;5;28mself\u001B[39m)\n\u001B[1;32m   1062\u001B[0m \u001B[38;5;66;03m# Check data\u001B[39;00m\n\u001B[0;32m-> 1063\u001B[0m X \u001B[38;5;241m=\u001B[39m \u001B[38;5;28mself\u001B[39m\u001B[38;5;241m.\u001B[39m_validate_X_predict(X)\n\u001B[1;32m   1065\u001B[0m \u001B[38;5;66;03m# Assign chunk of trees to jobs\u001B[39;00m\n\u001B[1;32m   1066\u001B[0m n_jobs, _, _ \u001B[38;5;241m=\u001B[39m _partition_estimators(\u001B[38;5;28mself\u001B[39m\u001B[38;5;241m.\u001B[39mn_estimators, \u001B[38;5;28mself\u001B[39m\u001B[38;5;241m.\u001B[39mn_jobs)\n",
      "File \u001B[0;32m~/anaconda3/envs/DS3500/lib/python3.12/site-packages/sklearn/ensemble/_forest.py:636\u001B[0m, in \u001B[0;36mBaseForest._validate_X_predict\u001B[0;34m(self, X)\u001B[0m\n\u001B[1;32m    633\u001B[0m \u001B[38;5;250m\u001B[39m\u001B[38;5;124;03m\"\"\"\u001B[39;00m\n\u001B[1;32m    634\u001B[0m \u001B[38;5;124;03mValidate X whenever one tries to predict, apply, predict_proba.\"\"\"\u001B[39;00m\n\u001B[1;32m    635\u001B[0m check_is_fitted(\u001B[38;5;28mself\u001B[39m)\n\u001B[0;32m--> 636\u001B[0m \u001B[38;5;28;01mif\u001B[39;00m \u001B[38;5;28mself\u001B[39m\u001B[38;5;241m.\u001B[39mestimators_[\u001B[38;5;241m0\u001B[39m]\u001B[38;5;241m.\u001B[39m_support_missing_values(X):\n\u001B[1;32m    637\u001B[0m     force_all_finite \u001B[38;5;241m=\u001B[39m \u001B[38;5;124m\"\u001B[39m\u001B[38;5;124mallow-nan\u001B[39m\u001B[38;5;124m\"\u001B[39m\n\u001B[1;32m    638\u001B[0m \u001B[38;5;28;01melse\u001B[39;00m:\n",
      "\u001B[0;31mIndexError\u001B[0m: list index out of range"
     ]
    }
   ],
   "execution_count": 30
  },
  {
   "cell_type": "code",
   "metadata": {
    "ExecuteTime": {
     "end_time": "2024-12-01T20:07:18.772465Z",
     "start_time": "2024-12-01T20:07:18.765378Z"
    }
   },
   "source": [
    "# getting importance of each feature in the model\n",
    "\n",
    "# Access the trained Random Forest model from the pipeline\n",
    "rf_model_trained = pipeline.named_steps['regressor']\n",
    "\n",
    "# Get feature importances\n",
    "feature_names = pipeline.named_steps['preprocessor'].transformers_[0][1].get_feature_names_out(categorical_features)\n",
    "all_features = list(feature_names) + numerical_features\n",
    "feature_importances = rf_model_trained.feature_importances_\n",
    "\n",
    "# Combine feature names and their importances into a DataFrame\n",
    "import pandas as pd\n",
    "feature_importance_df = pd.DataFrame({\n",
    "    'Feature': all_features,\n",
    "    'Importance': feature_importances\n",
    "}).sort_values(by='Importance', ascending=False)\n",
    "\n",
    "# Display the feature importances\n",
    "print(feature_importance_df)\n"
   ],
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "                                Feature  Importance\n",
      "0                         cab_type_Uber         0.0\n",
      "15        destination_Boston University         0.0\n",
      "27                                 hour         0.0\n",
      "26                     surge_multiplier         0.0\n",
      "25                             distance         0.0\n",
      "24                 destination_West End         0.0\n",
      "23         destination_Theatre District         0.0\n",
      "22            destination_South Station         0.0\n",
      "21  destination_Northeastern University         0.0\n",
      "20            destination_North Station         0.0\n",
      "19                destination_North End         0.0\n",
      "18         destination_Haymarket Square         0.0\n",
      "17       destination_Financial District         0.0\n",
      "16                   destination_Fenway         0.0\n",
      "14              destination_Beacon Hill         0.0\n",
      "1                       source_Back Bay         0.0\n",
      "13                 destination_Back Bay         0.0\n",
      "12                      source_West End         0.0\n",
      "11              source_Theatre District         0.0\n",
      "10                 source_South Station         0.0\n",
      "9        source_Northeastern University         0.0\n",
      "8                  source_North Station         0.0\n",
      "7                      source_North End         0.0\n",
      "6               source_Haymarket Square         0.0\n",
      "5             source_Financial District         0.0\n",
      "4                         source_Fenway         0.0\n",
      "3              source_Boston University         0.0\n",
      "2                    source_Beacon Hill         0.0\n",
      "28                          day_of_week         0.0\n"
     ]
    }
   ],
   "execution_count": 31
  },
  {
   "metadata": {},
   "cell_type": "code",
   "outputs": [],
   "execution_count": null,
   "source": ""
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": ".venv",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.12.6"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}
